## go to your system terminal and type ``ollama serve`` 

## start with a simple example where mostly everything is in the notebook

here we have a chromadb one ollama server and  a story  written by yours truely [`Perplexity`](https://www.perplexity.ai/)

and a set of questions also related to the story to test the rag

In [ ]:
questions = [
"What was the main reason Lily decided to explore the hidden garden?",
"How did Jack describe the flowers in the garden?",
"What personal struggles did Lily and Jack share with each other?",
"What did the owl say when it first appeared to Lily and Jack?",
"What wish did Lily make for Jack?",
"How did the flowers react when Lily and Jack danced around them?",
"What did the owl mean by saying 'kindness must be shared'?",
"How did the setting of the garden contribute to the story?",
"What emotions did Jack express after Lily made her wish?",
"How did the relationship between Lily and Jack develop throughout the story?"
]


# don't run this code  as the variable is same in later code python will get confused 
# use the question manually of use your own list/ variable to call this list

## Below there is a story 

It will be sent to the **RAG** pipeline as a list of sentences toekenized by nltk "punkt" and "punkt_tab"
there will be a basic chromadb client set up and running at localhost:8000
there will be a  random collection created for storing the data

In [24]:
import ollama
import chromadb
import random 
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download("punkt_tab")
story="""

## The Hidden Garden.

In a small town, there was a garden hidden behind a tall wall. The townsfolk often whispered about it, saying it was enchanted. No one had seen the inside, but everyone had their own stories.

Some said it was filled with flowers that could sing, while others believed it was home to a wise old owl that could grant wishes. 

One sunny afternoon, a curious girl named Lily decided to explore the garden. 

Lilly had always been fascinated by the stories and wanted to see if they were true. 

As she approached the wall, she noticed a small, rusty gate slightly ajar. 

With her heart racing, she pushed it open and stepped inside.

The garden was more beautiful than she had imagined. 

Colorful flowers bloomed everywhere, and the air was filled with sweet scents. 

In the center stood an ancient oak tree, its branches stretching wide, offering shade to a small stone bench. 

As she walked closer, she noticed a figure sitting on the bench—a boy about her age, with messy brown hair and bright green eyes.

“Hi, I’m Jack,” he said, smiling. “I come here to think.”

“Hi, I’m Lily,” she replied, feeling a strange connection with him. “I’ve heard so many stories about this place.”

Jack nodded. “Most of them are true. The flowers do sing, but only when they feel happy. And the owl? He’s real too. He watches over the garden.”

Lily’s eyes widened. “Can we see him?”

Jack hesitated. “He only appears when he senses someone who truly believes in magic.”

Lily felt a spark of determination. “I believe! Let’s find him!”

Together, they explored the garden, laughing and sharing stories. They discovered a patch of flowers that began to hum softly when they danced around them. The more they danced, the louder the flowers sang, filling the air with joy.

As they played, they talked about their lives. Lily shared how she often felt lonely at school, while Jack revealed that he had a sick sister who couldn’t leave their home. They both understood what it felt like to wish for something more.

Suddenly, the air grew still, and a soft hoot echoed through the garden. They turned to see a majestic owl perched on a low branch of the oak tree. Its feathers shimmered in the sunlight, and its eyes sparkled with wisdom.

“Welcome, young ones,” the owl said in a deep, soothing voice. “You have shown kindness and belief. What is it that you seek?”

Lily and Jack exchanged glances, unsure of what to say. Finally, Lily spoke up. “I wish for Jack’s sister to get better.”

The owl nodded slowly. “True friendship and selflessness are the greatest forms of magic. I will grant your wish, but remember, kindness must be shared.”

With a flap of its wings, the owl vanished into the air, leaving behind a soft glow. Jack looked at Lily, tears of gratitude in his eyes. “Thank you, Lily. You’ve given me hope.”

As the sun began to set, they promised to meet again in the garden. They had found not only magic but also a friendship that would last a lifetime.



"""
#
documents = sent_tokenize(story)
for i in range(len(documents)):
    documents[i] = documents[i].replace('\n', '')

#print(documents)
j=random.randint(0,29999)

client = chromadb.HttpClient(host='localhost', port=8000)
collection = client.create_collection(name=f"docs{j}")

# store each document in a vector embedding database
for i, d in enumerate(documents):
  response = ollama.embeddings(model="mxbai-embed-large", prompt=d)
  embedding = response["embedding"]
  collection.add(
    ids=[str(i)],
    embeddings=[embedding],
    documents=[d]
  )

[nltk_data] Downloading package punkt to /home/riju279/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/riju279/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# Now we will do the real RAG

In [25]:
import pandas as pd
# an example prompt

prompt = "How did Jack describe the flowers in the garden?"
n_results=10
# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt,
  model="mxbai-embed-large"
)
results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=n_results
)
print("-------\n\n The Retrieved data is: \n\n",results['documents'][0][:int(n_results-1)],"\n\n\n--------")

data = [doc[0:] for doc in results['documents'][:int(n_results-1)]]

# Now 'data' contains the 3 most relevant documents
#print(f"context is \n\n {data} \n\n")

# generate a response combining the prompt and data we retrieved in step 2
LM="mistral-nemo:latest"

output = ollama.generate(
  model=LM,
  prompt=f"Using this data: {data}. Respond to this prompt: {prompt}"
)

print("The Answer is:\n\n ------------\n\n",output['response'],f"\n\n the answer is given by: {LM}")

-------

 The Retrieved data is: 

 ['He watches over the garden.”Lily’s eyes widened.', 'Let’s find him!”Together, they explored the garden, laughing and sharing stories.', 'The garden was more beautiful than she had imagined.', 'Some said it was filled with flowers that could sing, while others believed it was home to a wise old owl that could grant wishes.', 'Jack looked at Lily, tears of gratitude in his eyes.', 'They discovered a patch of flowers that began to hum softly when they danced around them.', '“I’ve heard so many stories about this place.”Jack nodded.', '## The Hidden Garden.', 'Colorful flowers bloomed everywhere, and the air was filled with sweet scents.'] 


--------
The Answer is:

 ------------

 Jack didn't directly describe the flowers in his dialogue or thoughts within the provided data. However, it is mentioned that "Colorful flowers bloomed everywhere" and "Some said it was filled with flowers that could sing." So, based on these descriptions, Jack likely saw t

### Whatever we saw here, was the basic lag Pipeline without any involvement of DSPY

now comes the magic

## Working example of a simple **RAG** where embedding and retrieval is managed by ollama

make a dump file name it `data.md`,   in the same directory you are running this code.
else use an absolute path

In [27]:
import random
import dspy
import ollama
import chromadb
import nltk
from nltk.tokenize import sent_tokenize

### Press Ctrl + Shift + A after selecting the bottom section to get those modules
""" nltk.download('punkt')
nltk.download("punkt_tab") """

# Initialize a random collection name
j = random.randint(0, 29999)

# Create a ChromaDB client
client = chromadb.HttpClient(host='localhost', port=8000)
collection = client.create_collection(name=f"docs{j}")

# Define the path to the Markdown file
markdown_file_path = str(input("Please enter the absolute path to your markdown file"))  # Update this path to your Markdown file

# Read the Markdown file and store its content in a string variable named 'file'
with open(markdown_file_path, 'r', encoding='utf-8') as file:
    file_content = file.read()


# Store documents in the vector embedding database
documents = sent_tokenize(file_content)  # Replace with your actual documents

# made an embedding function
def embedOllama(texts):
    for i, d in enumerate(texts):
        response = ollama.embeddings(model="mxbai-embed-large", prompt=d)
        embedding = response["embedding"]
        collection.add(
        ids=[str(i)],
        embeddings=[embedding],
        documents=[d]
    )

# called embedOllama on documents
embedOllama(documents)

# Function to retrieve relevant documents based on a query
def retrieve_documents(prompt, n_results=10):
    # Generate an embedding for the prompt
    response = ollama.embeddings(
        prompt=prompt,
        model="mxbai-embed-large"
    )
    
    # Query the collection for the most relevant documents
    results = collection.query(
        query_embeddings=[response["embedding"]],
        n_results=n_results
    )
    
    # Print the top results
    #print(results['documents'][0][:int(n_results)], "\n\n\n--------")
    
    # Prepare data for further processing
    data = [doc[0:] for doc in results['documents'][:int(n_results-1)]]
    return data

# Example usage
""" prompt = "describe AI in Enhanced Customer Service"
retrieved_docs = retrieve_documents(prompt,2) """

#######################################################
#        #### working code for retirval ####          #
#######################################################





olm=dspy.OpenAI(api_base="http://localhost:11434/v1/", api_key="ollama", model="mistral-nemo:latest", stop='\n\n', model_type='chat')



class GenerateAnswer(dspy.Signature):
    """Answer questions with detailed factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 5 and 10 words")


class GenerateSearchQuery(dspy.Signature):
    """Write a simple search query that will help answer a complex question."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    query = dspy.OutputField()


generate_answer = dspy.ChainOfThought(GenerateAnswer)

#generate_answer=dspy.ChainOfThought(GenerateSearchQuery)

# Call the predictor on a particular input alongside a hint.


question='How did Jack describe the flowers in the garden?'
# pre query db for context
context=retrieve_documents(question,5)
# send question and context to db

prompt=str(f"tell me why{[question,context]}")
# post query db for better answer


retrieved_docs = retrieve_documents(question,n_results=3) #  (GenerateSearchQuery) ## changed
dspy.configure(lm=olm,rm=retrieved_docs)


pred = generate_answer(question=question,context=context)

print(f"\n\n\n The answer is:    {pred['answer']} \n\n because: {pred['rationale']} \n\n The long form context is {context}")




 The answer is:    Beautifully singing 

 because: produce the answer. We need to find where Jack speaks about the flowers in the garden. The context mentions "flowers that could sing," but it doesn't specify who said this. However, since we're looking for how Jack described them, and there's no direct quote from him, we can infer based on the overall description of the garden 

 The long form context is [['He watches over the garden.”\n\nLily’s eyes widened.', 'Let’s find him!”\n\nTogether, they explored the garden, laughing and sharing stories.', 'The garden was more beautiful than she had imagined.', 'Some said it was filled with flowers that could sing, while others believed it was home to a wise old owl that could grant wishes.', 'Jack looked at Lily, tears of gratitude in his eyes.']]
